## 1. アプリデータのタイトル列クリーンアップと整形
- 表記ゆれ対策のためのタイトル前処理

### 1.1. ライブラリとファイルの読み込み

In [7]:
import pandas as pd
import re

# 入力ファイルと出力ファイルのパス
input_path = "gamei_apps_sales_summary_utf8bom_ver3.csv"
output_full_path = "full_data_with_cleaned_titles.csv"
output_titles_only_path = "cleaned_unique_titles.csv"

# 元データの読み込み
df = pd.read_csv(input_path)

### 1.2. タイトルクリーンアップ関数の定義

In [8]:
def clean_title(title):
    if pd.isna(title):
        return None
    # あらゆる括弧とその中の文字列を削除（半角＋全角）
    return re.sub(r'[（\(\[【［][^）\)\]】］]*[）\)\]】］]', '', str(title)).strip()

### 1.3. クリーンなタイトル列を追加

In [9]:
# 元データに「クリーンタイトル」列を追加
df["クリーンタイトル"] = df["タイトル"].map(clean_title)

### 1.4. クリーンタイトル付きの元データを保存

In [10]:
# 元データ＋クリーンタイトル列を含めて保存
df.to_csv(output_full_path, index=False, encoding="utf-8-sig")
print(f"元データにクリーンタイトル列を加えて保存：{output_full_path}")

元データにクリーンタイトル列を加えて保存：full_data_with_cleaned_titles.csv


### 1.5. クリーンタイトルのみ抽出して保存

In [11]:
# クリーンタイトルのユニーク値のみ抽出
unique_titles = df["クリーンタイトル"].dropna().unique()
cleaned_df = pd.DataFrame(unique_titles, columns=["タイトル"])

# タイトル一覧を保存（統合作業用）
cleaned_df.to_csv(output_titles_only_path, index=False, encoding="utf-8-sig")
print(f"クリーンなタイトル一覧を保存：{output_titles_only_path}（{len(cleaned_df)} 件）")

クリーンなタイトル一覧を保存：cleaned_unique_titles.csv（393 件）


### 補足：保存した2つのCSVの使い分け
- `full_data_with_cleaned_titles`.csv → 統合や元データ分析用
- `cleaned_unique_titles.csv` → 検索やスクレイピング入力用

## 2. アプリタイトルから発売日データを自動取得（スクレイピング）

### 2.1.　ライブラリのインポート

In [12]:
import time
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

### 2.2. 入出力ファイル・Chrome起動設定

In [13]:
# ▼ ファイルパス
input_file = "cleaned_unique_titles.csv"
output_file = "apps_release_dates.csv"

# ▼ Chrome起動設定（必要に応じてヘッドレスも可）
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 必要ならON

# ▼ WebDriver起動
driver = webdriver.Chrome(options=options)

### 2.3. データ読み込みと初期化

In [14]:
# ▼ 全件読み込み＆発売日列追加
df = pd.read_csv(input_file).copy()
df["発売日"] = None
total = len(df)
print(f"{total} 件のタイトルを処理します。")

393 件のタイトルを処理します。


### 2.4. メイン処理ループ

In [15]:
# ▼ メイン処理ループ
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{total}] 検索中：{title}")

    try:
        # トップページ → 検索実行
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # 検索結果の表示待機（タイムアウト対策あり）
        try:
            WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
            )
            links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        except TimeoutException:
            print(" → 検索結果が見つかりません（タイムアウト）")
            continue

        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]
        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            continue

        # ゲーム紹介ページから発売日取得
        release_date = None
        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")
                for th in th_list:
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            release_date = td.text.strip()
                            break
                        except:
                            continue

                if release_date:
                    df.at[idx, "発売日"] = release_date
                    print(f" → 発売日：{release_date}")
                    break

            except Exception as sub_e:
                print(" → ページ処理中エラー:")
                traceback.print_exc()
                continue

        if not release_date:
            print(" → 発売日が見つかりません")

    except Exception as e:
        print(" → メイン処理エラー:")
        traceback.print_exc()
        continue

[1/393] 検索中：スーパーガンダムロワイヤル
 → 発売日：2015/10/16
[2/393] 検索中：武器よさらば
 → 発売日：2017/03/30
[3/393] 検索中：スーパーロボット大戦X-Ω
 → 発売日：2015/10/05
[4/393] 検索中：FINAL FANTASY GRANDMASTERS
 → 発売日：2015/10/01
[5/393] 検索中：ドラゴンプロジェクト
 → 発売日：2016/06/03
[6/393] 検索中：戦の海賊
 → 発売日：2015/08/27
[7/393] 検索中：誰ガ為のアルケミスト
 → 発売日：2016/01/28
[8/393] 検索中：デッキヒーローズ-本格派戦略カードゲーム-
 → 検索結果が見つかりません（タイムアウト）
[9/393] 検索中：美少女戦士セーラームーン　セーラームーンドロップス
 → 発売日：2015/09/03
[10/393] 検索中：モンスターハンター エクスプロア
 → 発売日：2015/09/29
[11/393] 検索中：夢色キャスト
 → 発売日：2024/09/29
[12/393] 検索中：グリムノーツ
 → 発売日：2016/01/21
[13/393] 検索中：DAME×PRINCE -ダメ王子たちとのドタバタ恋愛ADV
 → 発売日：2016/03/31
[14/393] 検索中：かんぱにガールズ ファンタジーRPG
 → 検索結果が見つかりません（タイムアウト）
[15/393] 検索中：デジモンリンクス
 → 発売日：2016/03/24
[16/393] 検索中：BFBチャンピオンズ2.0
 → 発売日：2016/06/29
[17/393] 検索中：フェアリーテイル 極・魔法乱舞
 → 発売日：2015/12/04
[18/393] 検索中：12オーディンズ - 王道RPG
 → 検索結果が見つかりません（タイムアウト）
[19/393] 検索中：ラストピリオド - 終わりなき螺旋の物語 -
 → 発売日：2016/05/10
[20/393] 検索中：聖闘士星矢 ゾディアック ブレイブ
 → 発売日：2016/01/28
[21/393] 検索中：ブレイブファンタジア
 → 発売日：2016/01/08
[22/393] 検索中：新

### 2.5. 保存と終了処理

In [16]:
# ▼ ドライバ終了＆結果保存
driver.quit()
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n 処理完了 → {output_file} に保存されました。")


 処理完了 → apps_release_dates.csv に保存されました。
